In [3]:
import formulas
import storage
import pandas as pd
import os

if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_dir = website_creator_path + os.path.sep + 'order_files'
    docs_dir = storage.get_docs_path(website_creator_path)
    df_formulas =  storage.get_formulas(formula_file_path)
    sort_orders = storage.get_sort_orders(order_file_dir)
    
    storage.delete_directory_if_it_exists(docs_dir)

    #Create formula by year summary
    formula_by_year_summary_df = formulas.get_formula_by_year_summary_df(df_formulas)
    storage.create_sub_directories(docs_dir, formula_by_year_summary_df)
    storage.create_index_files(base_dir=docs_dir, folder_regex='.*/Formulas/.*', book_collapse=True, sort_orders=sort_orders)
    # formulas.create_formula_files(docs_dir, df_formulas)

    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(df_calculus, formulas.formulas_on_formula_sheet(df_formulas))
    # display(styler_calculus)

In [19]:
docs_dir

'/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/content/docs'

In [6]:
import re
filepath = website_creator_path + os.path.sep + 'sort_orders.csv'

def get_sort_orders(file_path):
    return(pd.read_csv(filepath))



    
# def get_sort_order(df_sort_orders, 

sort_orders = pd.read_csv(filepath)

docs_dir = '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/content/docs'
current_dir = docs_dir + '/NSW/Formulas/By year'

_convert_path_to_list(path = current_dir, exclude_regex = '^'+docs_dir)
# sort_orders.dropna(subset = sort_orders.columns[1])

# re.sub(pattern=docs_dir, repl='', string=current_dir)

['NSW', 'Formulas', 'By year']

In [24]:
import re

docs_dir = '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/content/docs'
current_dir = docs_dir + '/NSW/Formulas/By year'

# re.sub(pattern='^h', repl='', string='hello')

my_list = current_dir.split(sep=os.path.sep)
my_list = list(filter(lambda item:item!='', my_list))
my_list

['home',
 'charl',
 'Onedrive',
 'Documents_Charl',
 'Computer_Technical',
 'Programming_GitHub',
 'AustralianSchoolMaths',
 'content',
 'docs',
 'NSW',
 'Formulas',
 'By year']